In [30]:
#immitates the qubit

import numpy as np

def randbin(data, F):
    phi = data.const * F * data.t * data.F_degree
    p_0 = (math.sin(phi/2)) ** 2
    return np.random.choice([0,1], size=(1,1), p=[p_0, 1-p_0]).reshape(1)[0]

In [66]:
#information about the experiment

class data:
    
    const = 100000 #magnetic constant
    F_degree = 180 / math.pi #degrees to radians
    
    t = 0 #present time
    t_min = 3.6 * 10 ** (-9) #starts at
    t_max = 360 * 10 ** (-9) #finishes at
    
    #set the flux range
    
    flux_min = -1.2
    flux_max = 1.2 
    flux_step = 0.1
    n = 0 #a number of dots
    
    def __init__(self, present, Tmin, Tmax, Fmin, Fmax, step):
        data.t = present
        data.t_min = Tmin
        data.t_max = Tmax
        data.flux_min = Fmin
        data.flux_max = Fmax
        data.flux_step = step
        
        data.n = round((Fmax - Fmin)/step) + 1
    
    def step(self, x):
        data.t *= 2 ** x

In [85]:
#fills service arrays with initial values

def begin(data, prev, pres, init):
    for i in range(data.n):
        prev.append(1 / data.n)
        pres.append(1 / data.n)
        init.append(1 / data.n)

In [73]:
#calculates the standart deviation

def stdev(array):
    
    avg = 0
    for i in range(len(array)):
        avg += array[i]
    avg /= len(array)
    
    sum = 0
    for i in range(len(array)):
        sum += (array[i] - avg) ** 2
        
    #костыль !!!   
        
    if sum == 0:
        return 
    else:
        return math.sqrt(sum / len(array))

In [70]:
#fits the qubit's probability distribution

def prob(array, p, q):
    
    sigma = stdev(array)
    return math.exp( - (p - q) ** 2 / (math.sqrt(2) * sigma) ** 2) / (math.sqrt(2 * math.pi) * sigma)

In [54]:
#one iteration of bayesian learning algorithn

def bayes(data, p, prev, pres):
    
    normalization = 0 #normalization coefficient
    
    for i in range(len(prev)):
        normalization += prob(prev, p, prev[i]) * prev[i]
    
    for i in range(len(pres)):
        pres[i] = prob(prev, p, prev[i]) * pres[i] / normalization    

In [87]:
#main

t_min = 3.6 * 10 ** (-9) #starts at
t_max = 360 * 10 ** (-9) #finishes at
flux_min = -1.2 
flux_max = 1.2 #the flux range
flux_step = 0.1

a = data(t_min, t_min, t_max, flux_min, flux_max, flux_step)

previous = []
present = []
init = []

begin(a, previous, present, init)

print(present)

for i in range( int(math.log2(a.t_max / a.t_min)) ):
    a.step(i) #time now is multiplied by 2
    p = randbin(a, 0.001) #qubit's state is changed
    bayes(a, p, previous, present) #distribution is changed
    print(present)
    previous = present



[0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04]
[0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998]
[0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999

In [ ]:
#проблемы:
# 1) как запускать алгоритм, если на первом шаге распределение равномерное и среднеквадратичное отклонение = 0 (stdev)
# 2) знание констант для адекватной работы